In [173]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import copy
from datetime import date, datetime
from pyxirr import xirr

In [174]:
class Portfolio():
    def __init__(self, max_loss, interval, max_pos, budge):
        self.max_loss = max_loss
        self.interval = interval
        self.max_pos = max_pos
        self.position = []
        self.last_buy  = datetime(1949, 10, 1, 9, 31, 0)
        self.last_sell = datetime(1949, 10, 1, 9, 31, 0)
        self.press = 0
        self.trades = []
        self.cash = budge

In [175]:
class Regression():
    def __init__(self, boll_data, his_price, max_loss=(0.01, 0.02), interval=5, max_pos=1000, strategy = None):
        self.boll_data = boll_data
        self.his_price = his_price
        self.cash_per_pos = 10000
        self.strategy = strategy
        self.portfolio = Portfolio(max_loss, interval, max_pos, self.cash_per_pos)

    def forward(self, period='1m'):
        boll_data = self.boll_data
        his_price = self.his_price

        for current_time, price_row in his_price.iterrows():
            boll_row = boll_data[boll_data['date'] == current_time.strftime('%Y-%m-%d')]
            self.strategy._eval(current_time, self.portfolio, boll_row, price_row)
            
        return self.portfolio
        


In [192]:
class MidBoll():
    def __init__(self):
        pass
        
    def _eval(self, curr_time, portfolio, boll_row, price_row):
        interval = portfolio.interval
        max_pos = portfolio.max_pos
        
        if len(boll_row) == 0:
            # 非交易日
            return portfolio
            
        boll = boll_row['MID'].item()
        min_loss, max_loss = portfolio.max_loss
        bp = round(boll / (1 - max_loss), 3)
        bp_low = round(boll / (1 - min_loss), 3)
        sl = boll
        
        op, cp, hp, lp, vol, money = price_row.values
        trade = {
            'date': curr_time, 
            'sl': sl,
            'bp': bp,
            'op': op,
            'cp': cp,
            'hp': hp,
            'lp': lp,
            'vol': vol
        }
        # 选择价格
        price = cp
        if price <= sl:
            # 触发止损
            if len(portfolio.position) > 0:
                # print(f"{curr_time} sell\t{sl, bp}\t{op, cp, hp, lp}")
                for buy_trade in portfolio.position:
                    pos = buy_trade['pos']
                    sell_trade = copy.deepcopy(trade)
                    sell_trade.update({
                        'op': 'sell',
                        'pos': pos, 
                        'money': pos * price,
                        'price': price, 
                    })
                    portfolio.trades.append({
                        'buy_trade': buy_trade,
                        'sell_trade': sell_trade
                    })
                portfolio.position = []
                portfolio.last_sell = curr_time
        elif sl < price and price <= bp and bp_low < price:
            # 价格在布林带区间，且价格在 bp_low 之上（防止频繁扫单），触发买入
            if (curr_time - portfolio.last_buy).days >= interval and len(portfolio.position) < max_pos:
                # print(f"{curr_time} buy \t{sl, bp}\t{op, cp, hp, lp}")
                # 仓位是 100 手的整数倍
                pos = portfolio.cash // (price * 100) * 100
                trade.update({
                    'op': 'buy',
                    'pos': pos, 
                    'money': pos * price,
                    'price': price, 
                })
                portfolio.position.append(trade)
                portfolio.last_buy = curr_time
                portfolio.press = max(portfolio.press, len(portfolio.position))
        elif bp < price:
            # 等待机会
            pass
    
        return portfolio

def save(portfolio, start_time, end_time, nk):
    op, cp, hp, lp, vol, money = his_price.loc[end_time].values
    trade = {
        'date': end_time, 
        'sl': None,
        'bp': None,
        'op': op,
        'cp': cp,
        'hp': hp,
        'lp': lp,
        'vol': vol
    }
    for buy_trade in portfolio.position:
        pos = buy_trade['pos']
        sell_trade = copy.deepcopy(trade)
        sell_trade.update({
            'op': 'sell',
            'pos': pos, 
            'money': pos * cp,
            'price': cp, 
        })
        portfolio.trades.append({
            'buy_trade': buy_trade,
            'sell_trade': sell_trade
        })
        
    trades = pd.json_normalize(portfolio.trades)
    
    trades['profit'] = trades['sell_trade.money'] - trades['buy_trade.money']
    profit = trades['profit'].sum().round(2)
    
    buy_dates = trades['buy_trade.date']
    buy_amounts = trades['buy_trade.money'] * -1

    sell_dates = trades['sell_trade.date']
    sell_amounts = trades['sell_trade.money']

    dates = pd.concat([buy_dates, sell_dates])
    amounts = pd.concat([buy_amounts, sell_amounts])
    
    rate = round(xirr(dates, amounts) * 100, 2)
    apr = round(xirr([start_time, end_time], [-10000 * portfolio.press, 10000 * portfolio.press + profit]) * 100, 2)
    
    trades = trades[['buy_trade.date', 'buy_trade.sl','buy_trade.pos', 'buy_trade.price', 'buy_trade.money', \
                     'sell_trade.date', 'sell_trade.sl', 'sell_trade.pos', 'sell_trade.price', 'sell_trade.money', 'profit']]


    max_loss = str(portfolio.max_loss)
    interval = str(portfolio.interval)
    
    # trades.to_csv(f'data-{max_loss}-{interval}.csv')

    return {
        'nk': nk,
        'max_loss': portfolio.max_loss,
        'interval': portfolio.interval,
        'max_pos': portfolio.max_pos,
        'press': portfolio.press,
        'trades': len(trades),
        'profit': profit,
        'xirr': rate,
        'apr': apr
    }, trades


In [177]:
def year_print(trade0):
    buy_trade  = trade0[['buy_trade.date', 'buy_trade.pos', 'buy_trade.price', 'buy_trade.money']]
    sell_trade = trade0[['sell_trade.date', 'sell_trade.pos', 'sell_trade.price', 'sell_trade.money']]
    
    # Acquired（持股增加）或Disposed（持股减少）
    buy_trade = buy_trade.rename(columns={'buy_trade.date': 'date', 'buy_trade.pos': 'pos', 'buy_trade.price': 'price', 'buy_trade.money': 'money'})
    buy_trade['op'] = 'Purchase'
    sell_trade = sell_trade.rename(columns={'sell_trade.date': 'date', 'sell_trade.pos': 'pos', 'sell_trade.price': 'price', 'sell_trade.money': 'money'})
    sell_trade['op'] = 'Sell'
    
    trade0 = pd.concat([buy_trade, sell_trade])
    trade0 = trade0.sort_values(by='date')
    
    principal = 10000 * ovw['press']
    pos = 0
    
    account = {
        'cash': principal,
        'pos': 0
    }
    mv_list = []
    for _date, price_row in his_price.iterrows():
        price = price_row['close'].item()
        date_buy = trade0[(trade0['date'] == _date) & (trade0['op'] == 'Purchase')]
        date_sell = trade0[(trade0['date'] == _date) & (trade0['op'] == 'Sell')]
        
        buy_pos = date_buy['pos'].sum()
        sell_pos = date_sell['pos'].sum()
    
        buy_money = date_buy['money'].sum()
        sell_money = date_sell['money'].sum()
    
        account['cash'] += sell_money - buy_money
        account['pos'] += buy_pos - sell_pos
    
        cash = account['cash']
        market = account['pos'] * price
        mv = {
            'date': _date,
            'cash': cash,
            'market': market,
            'total': cash + market
        }
        mv_list.append(mv)
    
    aaa = pd.json_normalize(mv_list)
    
    for year in range(2013, 2026):
        year_df = aaa[(aaa['date'] >= datetime(year, 1, 1, 0, 0, 0)) & (aaa['date'] <= datetime(year, 12, 31, 23, 0, 0))]
        b = year_begin = year_df.head(1)['total'].item()
        e = year_end   = year_df.tail(1)['total'].item()
        profit = round(e - b, 2)
        rate = round(profit * 100 / b, 2)
        print(f'{year}: {rate}, {profit}')

In [188]:
def load_his_price():
    his_price_raw = ak.fund_etf_hist_em(symbol="518880", period="daily", start_date="20130729", end_date="20250428", adjust="")
    his_price_raw = his_price_raw.rename(columns=col_mapping)[col_mapping.values()]
    his_price_raw["date_idx"] = pd.to_datetime(his_price_raw["date"])
    his_price = his_price_raw.drop(columns='date')
    his_price = his_price.set_index("date_idx")
    
    # his_price = pd.read_csv('../joinQuant/data-20240119-2025-01-23.csv', index_col=0, parse_dates=True)
    return his_price

In [189]:
def load_boll_data(n, k):
    boll_raw = ak.fund_etf_hist_em(symbol="518880", period="daily", start_date="20130729", end_date="20250428", adjust="")
    
    boll_raw['std'] = boll_raw['收盘'].rolling(window=n).std(ddof=1).round(3)
    boll_raw['MID'] = boll_raw['收盘'].rolling(window=n).mean().round(3)
    boll_raw['UP']  = boll_raw['MID'] + k * boll_raw['std']
    boll_raw['LOW'] = boll_raw['MID'] - k * boll_raw['std']
    
    col_mapping = {'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'vol', '成交额': 'money'}
    
    boll = boll_raw.rename(columns=col_mapping)
    return boll

In [202]:
his_price = load_his_price()
start_time = his_price.index[0]
end_time = his_price.index[-1]

boll_nk_list  = [(20, 2), (30, 2), (50, 2), (100, 2), (150, 2)]
lr_list       = [(0, 0.02)] #[(0,0.01), (0, 0.02), (0, 0.05), (0, 0.1)]
interval_list = [0] #[0, 3, 5, 10]
max_pos_list  = [10] # [1, 2, 5, 10]

report_list = []
trade_list= []
for i in range(2, 16):
    nk = (i * 10, 2)
    boll = load_boll_data(*nk)
    for lr in lr_list:
        for interval in interval_list:
            for max_pos in max_pos_list:
                regression = Regression(boll, his_price, max_loss=lr, interval=interval, max_pos=max_pos, strategy=MidBoll())
                portfolio = regression.forward()
                ovw, t = save(copy.deepcopy(portfolio), start_time, end_time, nk)
                report_list.append(ovw)
                trade_list.append(t)

In [204]:
report = pd.json_normalize(report_list)
report = report.sort_values(by='xirr', ascending=False)
report.head(20)

,nk,max_loss,interval,max_pos,press,trades,profit,xirr,apr
2,"(40, 2)","(0, 0.02)",0,10,10,584,75186.1,16.35,4.88
1,"(30, 2)","(0, 0.02)",0,10,10,702,59426.6,14.72,4.05
4,"(60, 2)","(0, 0.02)",0,10,10,424,76067.9,12.63,4.93
6,"(80, 2)","(0, 0.02)",0,10,10,318,84067.5,12.46,5.33
9,"(110, 2)","(0, 0.02)",0,10,10,287,90510.9,12.01,5.64
12,"(140, 2)","(0, 0.02)",0,10,10,248,82798.4,11.95,5.26
11,"(130, 2)","(0, 0.02)",0,10,10,271,91008.3,11.84,5.66
5,"(70, 2)","(0, 0.02)",0,10,10,390,73840.0,11.80,4.82
8,"(100, 2)","(0, 0.02)",0,10,10,289,88585.0,11.61,5.54
0,"(20, 2)","(0, 0.02)",0,10,10,869,44117.8,11.60,3.16


In [185]:
trade = trade_list[3]
year_print(trade)
print(trade.shape)
trade[trade['profit'] < 0].shape

2013: 0.0, 0.0
2014: -0.77, -772.8
2015: -2.79, -2764.0
2016: 12.14, 11713.9
2017: -1.19, -1290.1
2018: -1.01, -1079.6
2019: 6.95, 7365.9
2020: -2.79, -3161.9
2021: -1.97, -2171.8
2022: 1.22, 1320.9
2023: 4.94, 5390.0
2024: 22.66, 26020.7
2025: 22.44, 31843.2
(213, 11)


(168, 11)